In [1]:
import json
import pandas as pd
import numpy as np
import re
import copy
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [2]:
data = [] 
for line in open('review_train.json', 'r'): 
    data.append(json.loads(line))

In [3]:
dat = data[0:1000000]

In [4]:
reviewslist = copy.deepcopy(dat)

In [5]:
REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])|(\$)|(\*)|(\%)|(\_)|(\=)|(\#)|(\&)|(\~)|(\@)")#[^\P{P}-]+
REPLACE_WITH_SPACE = re.compile("(\n)|(\-)|(\/)|(\d)")

def preprocess_reviews(reviews):
    reviews = REPLACE_NO_SPACE.sub("", reviews)
    reviews = REPLACE_WITH_SPACE.sub(" ", reviews)
    
    return reviews

reviewsTEXT_clean = copy.deepcopy(reviewslist)

for ind in range(len(reviewslist)):
    texts = ''
    texts = reviewslist[ind]['text']
    texts = texts.lower()
    texts = re.sub('isnt','isn\'t', texts)
    texts = re.sub('wasnt','wasn\'t', texts)
    texts = re.sub('werent','weren\'t', texts)
    texts = re.sub('dont','don\'t', texts)
    texts = re.sub('doesnt','doesn\'t', texts)
    texts = re.sub('didnt','didn\'t', texts)
    texts = re.sub('hasnt','hasn\'t', texts)
    texts = re.sub('havent','haven\'t', texts)
    texts = re.sub('hadnt','hadn\'t', texts)
    texts = re.sub('mightnt','mightn\'t', texts)
    texts = re.sub('shouldnt','shouldn\'t', texts)
    texts = re.sub('isn','isn\'t', texts)
    texts = re.sub('wasn','wasn\'t', texts)
    texts = re.sub('weren','weren\'t', texts)
    texts = re.sub('don','don\'t', texts)
    texts = re.sub('doesn','doesn\'t', texts)
    texts = re.sub('didn','didn\'t', texts)
    texts = re.sub('hasn','hasn\'t', texts)
    texts = re.sub('haven','haven\'t', texts)
    texts = re.sub('hadn','hadn\'t', texts)
    texts = re.sub('mightn','mightn\'t', texts)
    texts = re.sub('shouldn','shouldn\'t', texts)
    
    texts = re.sub('n\'t',' not', texts)
    
    #add NOT_
    pattern = r'\.|\;|\!|\?|\,|\)|\(|\:|\'|\"|\%'
    list_text=re.split(pattern,texts)
    
    sent=''
    for i in range(len(list_text)):
        list_text[i] = re.sub('\+','', list_text[i])
        list_text[i] = re.sub('\*','', list_text[i])
        list_text[i] = re.sub('\$','', list_text[i])
        list_text[i] = re.sub('\[','', list_text[i])
        list_text[i] = re.sub('\]','', list_text[i])
        list_text[i] = re.sub('\%','', list_text[i])
        list_text[i] = re.sub('\\\\',' ', list_text[i])
        matchObj1 = re.search(r'(.*)not (.*)',list_text[i])
        matchObj2 = re.search(r'(.*) never (.*)',list_text[i])
        
        if matchObj1 != None :
            sub=re.sub(r' ', " notadd", matchObj1.group(2))
            list_text[i] = re.sub(matchObj1.group(2) ,sub, list_text[i])
            list_text[i] = re.sub(r'not ','', list_text[i],1)
            #list_text[i] = re.sub(',',' ', matchObj1.group(3))
            sent = sent + list_text[i] + ' '
        
        elif matchObj2 != None :
            sub=re.sub(r' ', " notadd", matchObj2.group(2))
            list_text[i] = re.sub(matchObj2.group(2) ,sub, list_text[i])
            list_text[i] = re.sub(r'never ','', list_text[i],1)
            #list_text[i] = re.sub(',',' ', matchObj1.group(3))
            sent = sent + list_text[i] + ' '
        
        else:
            sent = sent + list_text[i] + ' '
   
    #print(sent)
    #print(texts)
    #print(texts)
    text_clean=preprocess_reviews(sent)
    #print(text_clean)
    reviewsTEXT_clean[ind]['text']=text_clean
    reviewsTEXT_clean[ind]['text_split']=text_clean.split()
    reviewsTEXT_clean[ind]['freq']=nltk.FreqDist(reviewsTEXT_clean[ind]['text_split'])
    
#reviewsTEXT_clean[0]

In [6]:
with open("review_train_pre.json", 'w') as fp:
        json.dump(reviewsTEXT_clean, fp)

In [7]:
#reviewsTEXT_clean=[]
#for line in open('review_train_pre.json', 'r'): 
#    reviewsTEXT_clean.append(json.loads(line))

In [8]:
#select words due to frequency order
AllFreq = reviewsTEXT_clean[0]['freq']
for ind in range(len(reviewsTEXT_clean)-1):
    freqs = reviewsTEXT_clean[ind+1]['freq']
    AllFreq.update(freqs)
    
wordList = sorted(AllFreq.items(),key=lambda item:item[1],reverse=True)

#choose the important ones--No.1-100
WORDS=[x[0] for x in wordList]
WORDS=WORDS[0:2000]

In [9]:
stop_words = set(stopwords.words('english')) 
negWords=set(["wouldn't",'isn','wasn',"weren't", "haven't", "hasn't", "couldn't", "isn't", 'doesn','hasn',"mustn't", 'mightn', 'shan', 'no', "wasn't",'aren', "didn't", "hadn't","don't",'nor',"won't",'weren',"doesn't","needn't", 'shouldn',"mightn't","shan't", 'wouldn',"shouldn't",'hadn'])
d=set(['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z','à'])
stopWords=stop_words-negWords
stopWords.update(d)

WORDS=[x for x in WORDS if x not in stopWords]

In [10]:
for i in range(len(reviewsTEXT_clean)):
    reviewsTEXT_clean[i]['text'] = ''
    for ind in reviewsTEXT_clean[i]['text_split']:
        if ind in WORDS:
            reviewsTEXT_clean[i]['text'] = reviewsTEXT_clean[i]['text'] + ind +' '
        else:
            reviewsTEXT_clean[i]['text'] = reviewsTEXT_clean[i]['text']

In [11]:
#create list of texts
texts= [x['text'] for x in reviewsTEXT_clean]

In [12]:
#vectorizer = CountVectorizer(token_pattern=r'\b[^\d\W]+\b')
vectorizer = CountVectorizer()
#X = vectorizer.fit_transform(texts)
transformer = TfidfTransformer()

tfidf = transformer.fit_transform(vectorizer.fit_transform(texts))

X=tfidf

In [13]:
#vectorizer.get_feature_names()
X_df=pd.DataFrame(X.A,columns=vectorizer.get_feature_names())

In [14]:
Reviews_df=X_df[WORDS]
Reviews_df

,food,good,place,great,service,like,time,would,get,one,...,band,organized,vet,melted,knowledge,organic,hospital,watched,reminded,dough
0,0.000000,0.000000,0.000000,0.000000,0.129006,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.00000,0.384155,0.000000,0.0,0.0
1,0.000000,0.043001,0.000000,0.043593,0.045170,0.048943,0.048280,0.000000,0.099067,0.099873,...,0.000000,0.000000,0.0,0.0,0.0,0.00000,0.000000,0.000000,0.0,0.0
2,0.000000,0.000000,0.000000,0.059664,0.000000,0.000000,0.000000,0.000000,0.067795,0.000000,...,0.000000,0.175691,0.0,0.0,0.0,0.00000,0.000000,0.000000,0.0,0.0
3,0.000000,0.080582,0.000000,0.081691,0.000000,0.000000,0.000000,0.092719,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.00000,0.000000,0.000000,0.0,0.0
4,0.000000,0.000000,0.000000,0.000000,0.025475,0.000000,0.163375,0.055809,0.055873,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.00000,0.000000,0.072729,0.0,0.0
5,0.074065,0.112462,0.037033,0.000000,0.039379,0.042668,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.00000,0.000000,0.000000,0.0,0.0
6,0.000000,0.000000,0.038054,0.000000,0.000000,0.175377,0.043250,0.044323,0.044373,0.089469,...,0.000000,0.000000,0.0,0.0,0.0,0.00000,0.000000,0.000000,0.0,0.0
7,0.157790,0.000000,0.078897,0.000000,0.083893,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.00000,0.000000,0.000000,0.0,0.0
8,0.000000,0.000000,0.194203,0.000000,0.000000,0.000000,0.000000,0.000000,0.226450,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.00000,0.000000,0.000000,0.0,0.0
9,0.000000,0.000000,0.052034,0.000000,0.000000,0.059951,0.059138,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.00000,0.000000,0.000000,0.0,0.0


In [15]:
#Add Business ID in the data frame
businessIDs=[x['business_id'] for x in reviewsTEXT_clean]
Reviews_df['business_id']=businessIDs
Reviews_df

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,food,good,place,great,service,like,time,would,get,one,...,organized,vet,melted,knowledge,organic,hospital,watched,reminded,dough,business_id
0,0.000000,0.000000,0.000000,0.000000,0.129006,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.00000,0.384155,0.000000,0.0,0.0,31292
1,0.000000,0.043001,0.000000,0.043593,0.045170,0.048943,0.048280,0.000000,0.099067,0.099873,...,0.000000,0.0,0.0,0.0,0.00000,0.000000,0.000000,0.0,0.0,35344
2,0.000000,0.000000,0.000000,0.059664,0.000000,0.000000,0.000000,0.000000,0.067795,0.000000,...,0.175691,0.0,0.0,0.0,0.00000,0.000000,0.000000,0.0,0.0,152538
3,0.000000,0.080582,0.000000,0.081691,0.000000,0.000000,0.000000,0.092719,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.00000,0.000000,0.000000,0.0,0.0,71871
4,0.000000,0.000000,0.000000,0.000000,0.025475,0.000000,0.163375,0.055809,0.055873,0.000000,...,0.000000,0.0,0.0,0.0,0.00000,0.000000,0.072729,0.0,0.0,64913
5,0.074065,0.112462,0.037033,0.000000,0.039379,0.042668,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.00000,0.000000,0.000000,0.0,0.0,112310
6,0.000000,0.000000,0.038054,0.000000,0.000000,0.175377,0.043250,0.044323,0.044373,0.089469,...,0.000000,0.0,0.0,0.0,0.00000,0.000000,0.000000,0.0,0.0,176386
7,0.157790,0.000000,0.078897,0.000000,0.083893,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.00000,0.000000,0.000000,0.0,0.0,139555
8,0.000000,0.000000,0.194203,0.000000,0.000000,0.000000,0.000000,0.000000,0.226450,0.000000,...,0.000000,0.0,0.0,0.0,0.00000,0.000000,0.000000,0.0,0.0,50173
9,0.000000,0.000000,0.052034,0.000000,0.000000,0.059951,0.059138,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.00000,0.000000,0.000000,0.0,0.0,13172


In [16]:
#Add stars in the data frame
stars=[x['stars'] for x in reviewsTEXT_clean]
Reviews_df['Stars']=stars
Reviews_df

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,food,good,place,great,service,like,time,would,get,one,...,vet,melted,knowledge,organic,hospital,watched,reminded,dough,business_id,Stars
0,0.000000,0.000000,0.000000,0.000000,0.129006,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.00000,0.384155,0.000000,0.0,0.0,31292,1.0
1,0.000000,0.043001,0.000000,0.043593,0.045170,0.048943,0.048280,0.000000,0.099067,0.099873,...,0.0,0.0,0.0,0.00000,0.000000,0.000000,0.0,0.0,35344,5.0
2,0.000000,0.000000,0.000000,0.059664,0.000000,0.000000,0.000000,0.000000,0.067795,0.000000,...,0.0,0.0,0.0,0.00000,0.000000,0.000000,0.0,0.0,152538,5.0
3,0.000000,0.080582,0.000000,0.081691,0.000000,0.000000,0.000000,0.092719,0.000000,0.000000,...,0.0,0.0,0.0,0.00000,0.000000,0.000000,0.0,0.0,71871,5.0
4,0.000000,0.000000,0.000000,0.000000,0.025475,0.000000,0.163375,0.055809,0.055873,0.000000,...,0.0,0.0,0.0,0.00000,0.000000,0.072729,0.0,0.0,64913,1.0
5,0.074065,0.112462,0.037033,0.000000,0.039379,0.042668,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.00000,0.000000,0.000000,0.0,0.0,112310,4.0
6,0.000000,0.000000,0.038054,0.000000,0.000000,0.175377,0.043250,0.044323,0.044373,0.089469,...,0.0,0.0,0.0,0.00000,0.000000,0.000000,0.0,0.0,176386,3.0
7,0.157790,0.000000,0.078897,0.000000,0.083893,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.00000,0.000000,0.000000,0.0,0.0,139555,1.0
8,0.000000,0.000000,0.194203,0.000000,0.000000,0.000000,0.000000,0.000000,0.226450,0.000000,...,0.0,0.0,0.0,0.00000,0.000000,0.000000,0.0,0.0,50173,3.0
9,0.000000,0.000000,0.052034,0.000000,0.000000,0.059951,0.059138,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.00000,0.000000,0.000000,0.0,0.0,13172,4.0


In [17]:
from pandas import DataFrame
DataFrame.to_csv(Reviews_df,"review_train_kaggle.csv",index=False)

In [18]:
#col=Reviews_df.columns.values

In [19]:
#DataFrame.to_csv(pd.DataFrame(col),"review_train_kaggle_colnames.csv",index=False)